### **Imports**

In [1]:
import time
start_time = time.time()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import string

### **DataSet** **Upload**

In [2]:
from google.colab import files , drive
drive.mount('/content/gdrive')
dataset = "/content/gdrive/My Drive/Colab Notebooks/covid_data.csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### **Dataframing**

In [3]:
df = pd.read_csv(dataset)

### **DataUpload check**

In [4]:
df.shape

(67166, 59)

In [5]:
df.head(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


In [6]:
df.nunique()

iso_code                                   192
continent                                    6
location                                   200
date                                       403
total_cases                              34313
new_cases                                 9391
new_cases_smoothed                       17775
total_deaths                             12928
new_deaths                                2305
new_deaths_smoothed                       5143
total_cases_per_million                  48425
new_cases_per_million                    27581
new_cases_smoothed_per_million           31234
total_deaths_per_million                 28379
new_deaths_per_million                    5548
new_deaths_smoothed_per_million           6782
reproduction_rate                          356
icu_patients                              1938
icu_patients_per_million                  3956
hosp_patients                             3696
hosp_patients_per_million                 6291
weekly_icu_ad

In [7]:
df.dtypes.value_counts()

float64    54
object      5
dtype: int64

In [8]:
df.dtypes

iso_code                                  object
continent                                 object
location                                  object
date                                      object
total_cases                              float64
new_cases                                float64
new_cases_smoothed                       float64
total_deaths                             float64
new_deaths                               float64
new_deaths_smoothed                      float64
total_cases_per_million                  float64
new_cases_per_million                    float64
new_cases_smoothed_per_million           float64
total_deaths_per_million                 float64
new_deaths_per_million                   float64
new_deaths_smoothed_per_million          float64
reproduction_rate                        float64
icu_patients                             float64
icu_patients_per_million                 float64
hosp_patients                            float64
hosp_patients_per_mi

In [9]:
"""
Loop for checking which features are object type
INPUT: pandas series
OUTPUT: list of objects
"""
list_of_objects = []
x=0
while x < 59:
  if df.dtypes.values[x] == object:
    list_of_objects.append(df.dtypes.index[x])
  x = x + 1
print(list_of_objects)

['iso_code', 'continent', 'location', 'date', 'tests_units']


In [10]:
df.tests_units.value_counts()

tests performed    21404
people tested       7417
samples tested      6128
units unclear        715
Name: tests_units, dtype: int64

### **Data Preprocessing** 

In [11]:
df.tests_units = df.tests_units.replace(['tests performed','people tested'],'Test Conducted')#tests conducted on people
df.tests_units = df.tests_units.replace(['units unclear'], 'Undefined')
df.tests_units.value_counts()

Test Conducted    28821
samples tested     6128
Undefined           715
Name: tests_units, dtype: int64

In [12]:
df.groupby(["tests_units", "continent"]).size()

tests_units     continent    
Test Conducted  Africa            3509
                Asia              6843
                Europe           11958
                North America     2825
                Oceania           1032
                South America     2654
Undefined       Africa             314
                Asia               401
samples tested  Africa            3020
                Asia              2130
                Europe             339
                North America      639
dtype: int64

In [13]:
correlation_matrix = df.corr().round(2)
correlation_matrix.shape #Large Martix difficlt to visualise

(54, 54)

In [14]:
df.corr()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
total_cases,1.000000,0.927408,0.943680,0.975554,0.873166,0.899021,0.089496,0.048408,0.059073,0.095975,0.036937,0.054495,0.001001,0.880238,0.259609,0.879452,0.175956,0.399555,-0.042680,0.543740,0.019060,0.862973,0.959524,0.138483,0.080678,0.889849,0.107264,0.080416,-0.044123,0.772965,0.785743,0.720548,0.766189,0.828894,-0.072406,-0.092761,-0.074817,-0.066546,0.069293,0.651471,-0.013659,0.028699,0.025096,0.021927,0.016082,-0.032055,-0.032795,0.022025,-0.003979,-0.010435,0.029893,-0.009621,0.018688,0.109533
new_cases,0.927408,1.000000,0.989372,0.938239,0.920075,0.919358,0.082449,0.080241,0.083312,0.089265,0.051701,0.066235,0.015581,0.894596,0.285105,0.900997,0.204854,0.444601,-0.038056,0.606716,0.040390,0.865643,0.822286,0.132813,0.098552,0.860167,0.123831,0.164753,-0.050241,0.631078,0.647362,0.565357,0.697322,0.704176,-0.071902,-0.095779,-0.074932,-0.064825,0.083729,0.726239,-0.015839,0.037958,0.036416,0.033023,0.023671,-0.037971,-0.039136,0.022412,0.003164,-0.009349,0.032950,-0.005597,0.025142,0.127935
new_cases_smoothed,0.943680,0.989372,1.000000,0.951382,0.911779,0.934130,0.084412,0.069948,0.084553,0.091333,0.048715,0.068914,0.011764,0.915514,0.291962,0.920151,0.209823,0.449988,-0.027653,0.611994,0.047473,0.862975,0.837623,0.135417,0.096035,0.876667,0.125837,0.168485,-0.051443,0.654256,0.670779,0.592636,0.695379,0.730342,-0.070754,-0.094710,-0.075638,-0.063646,0.084040,0.733613,-0.015985,0.038157,0.036582,0.033149,0.023734,-0.038214,-0.039339,0.022756,0.003033,-0.009538,0.033220,-0.005754,0.025225,0.130158
total_deaths,0.975554,0.938239,0.951382,1.000000,0.903376,0.930745,0.071585,0.037330,0.044373,0.111131,0.038812,0.052107,0.002222,0.888697,0.228291,0.905291,0.165805,0.429775,-0.091153,0.581862,-0.005737,0.840989,0.872250,0.110302,0.070422,0.850892,0.090141,0.175252,-0.059273,0.765228,0.777032,0.709352,0.758784,0.820101,-0.080904,-0.103936,-0.083785,-0.075232,0.078925,0.732813,-0.018100,0.034826,0.031090,0.028465,0.015330,-0.042337,-0.045866,0.027753,-0.004844,-0.011708,0.040843,-0.014393,0.026048,0.156502
new_deaths,0.873166,0.920075,0.911779,0.903376,1.000000,0.971187,0.059916,0.050936,0.054728,0.088988,0.094424,0.087409,0.011510,0.911191,0.358170,0.910825,0.268766,0.486762,-0.005623,0.707165,0.099565,0.717766,0.656328,0.083039,0.064481,0.682462,0.071795,0.304684,-0.062674,0.730979,0.744452,0.659240,0.768459,0.783057,-0.072901,-0.097728,-0.082955,-0.067142,0.121598,0.784289,-0.019520,0.038864,0.035326,0.032603,0.016242,-0.045011,-0.045039,0.029292,-0.002310,-0.006800,0.044435,-0.008877,0.026882,0.153321
new_deaths_smoothed,0.899021,0.919358,0.934130,0.930745,0.971187,1.000000,0.067983,0.048182,0.060137,0.092287,0.067659,0.096279,0.010369,0.958954,0.375636,0.954655,0.282490,0.502889,-0.031614,0.694009,0.068030,0.728056,0.700844,0.093259,0.061710,0.730113,0.078838,0.320405,-0.061485,0.755279,0.769057,0.691109,0.757173,0.813840

In [15]:
df.drop(['new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million'], axis=1, inplace=True)

In [16]:
"""
Loop to select the countries with the most common entries
INPUT: series of countries with number of entries
OUTPUT: list of the most common countries with the number of entries
"""

i = 0
theMostCommonCountry_List = []
while i < 10:
  part_index = str(df.location.value_counts().index[i])
  part_num_entries = str(df.location.value_counts()[i])
  theMostCommonCountry_List.append( part_index + ": " + part_num_entries )
  i = i + 1
print(theMostCommonCountry_List)

['Argentina: 403', 'Mexico: 403', 'Thailand: 400', 'Taiwan: 388', 'South Korea: 383', 'China: 382', 'United States: 382', 'World: 382', 'Japan: 382', 'Asia: 382']


In [27]:
i = 0
x = True
while x == True:
  if df.location[i] not in theMostCommonCountry_List:
    df.drop([i], inplace=True)
    i = i + 1
    print(i)
    if i == len(df.location):
      final_time = (time.time() - start_time)
      x = False

Streaming output truncated to the last 5000 lines.
28584
28585
28586
28587
28588
28589
28590
28591
28592
28593
28594
28595
28596
28597
28598
28599
28600
28601
28602
28603
28604
28605
28606
28607
28608
28609
28610
28611
28612
28613
28614
28615
28616
28617
28618
28619
28620
28621
28622
28623
28624
28625
28626
28627
28628
28629
28630
28631
28632
28633
28634
28635
28636
28637
28638
28639
28640
28641
28642
28643
28644
28645
28646
28647
28648
28649
28650
28651
28652
28653
28654
28655
28656
28657
28658
28659
28660
28661
28662
28663
28664
28665
28666
28667
28668
28669
28670
28671
28672
28673
28674
28675
28676
28677
28678
28679
28680
28681
28682
28683
28684
28685
28686
28687
28688
28689
28690
28691
28692
28693
28694
28695
28696
28697
28698
28699
28700
28701
28702
28703
28704
28705
28706
28707
28708
28709
28710
28711
28712
28713
28714
28715
28716
28717
28718
28719
28720
28721
28722
28723
28724
28725
28726
28727
28728
28729
28730
28731
28732
28733
28734
28735
28736
28737
28738
28739
28740
28741
2

In [29]:
print(final_time)

1858.6936254501343
